In [1]:
import os

# Set environment variables
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# import tensorflow as tf
# uncomment following lines if you are using TF2
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import json
import ast
import cv2
import argparse
from PIL import Image
import math
import sys
sys.path.append('./Mask_RCNN')
from mrcnn.config import Config
import mrcnn.model as modellib
import skimage.io
from collections import OrderedDict
from skimage.measure import find_contours, approximate_polygon
from xml_dumper import dump_as_cvat_annotation
import glob

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
tf.__version__

'1.15.4'

In [3]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 122 µs (started: 2021-05-02 11:19:48 +10:00)


In [4]:
# Run from the roadside/code directory:

# papermill -p IMAGE_FILE_PATH '../open-camera-test/*.jpg' -p OUTPUT_XML_PATH '../open-camera-test/detected_objects.xml' demo2-nb.ipynb temp-demo2-nb.ipynb

IMAGE_FILE_PATH = '../open-camera-test/*.jpg'                  # Path to one or more image files. Can include wildcards. See https://pymotw.com/2/glob/ for pattern matching details.
OUTPUT_XML_PATH = '../open-camera-test/detected_objects.xml'   # Path to output file which will contain metadata for detected objects.
TYPE = 'both'                                                  # what type of models to use [both,classes,v_shape]
#SKIP_NO = 1                                                    # int, num of frames to skip (must be >0)
#NUM_FRAMES = None                                              # how many frames to consider?
OD_MODEL = "inference_data/frozen_inference_graph_5classes.pb" # path to trained detection model
CLASSES_CVAT = "inference_data/5classes.csv"                   # classes you want to use for cvat, see readme for more details.
CLASSES_TYPE = "od"                                            # type of classes csv file [od, maskrcnn]
MASK_MODEL =  "inference_data/mask_rcnn_cvat_0160.h5"          # path to trained maskrcnn model
OD_THRESHOLD = 0.5                                             # threshold for IoU
MASK_THRESHOLD = 0.5                                           # threshold for maskrcnn
#SURVEY_TYPE = "v_shape"                                        # what to write in geojson [v_shape,classes]
TASK_ID = 0                                                    # required only if you want to use this in cvat
TASK_NAME = "demo"                                             # required only if you want to use this in cvat
DUMP_SQL = False

time: 588 µs (started: 2021-05-02 11:19:48 +10:00)


In [5]:
# Parameters
IMAGE_FILE_PATH = "../open-camera-test/*.jpg"
OUTPUT_XML_PATH = "../open-camera-test/detected_objects.xml"


time: 240 µs (started: 2021-05-02 11:19:48 +10:00)


In [6]:
class ObjectDetection:
    def __init__(self, model_path):
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(model_path , 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
                config = tf.ConfigProto()
                config.gpu_options.allow_growth=True
                self.sess = tf.Session(graph=self.detection_graph, config=config)

    def get_detections(self, image_np_expanded):
        image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')
        (boxes, scores, classes, num_detections) = self.sess.run([boxes, scores, classes, num_detections], feed_dict={image_tensor: image_np_expanded})
        return boxes, scores, classes, num_detections

    @staticmethod
    def process_boxes(boxes, scores, classes, labels_mapping, threshold, width, height):
        result = {}
        for i in range(len(classes[0])):
            if classes[0][i] in labels_mapping.keys():
                if scores[0][i] >= threshold:
                    xmin = int(boxes[0][i][1] * width)
                    ymin = int(boxes[0][i][0] * height)
                    xmax = int(boxes[0][i][3] * width)
                    ymax = int(boxes[0][i][2] * height)
                    label = labels_mapping[classes[0][i]]
                    if label not in result:
                        result[label] = []
                    result[label].append([xmin,ymin,xmax,ymax])
        return result

class Segmentation:
    def __init__(self, model_path, num_c=2):
        class InferenceConfig(Config):
            # Set batch size to 1 since we'll be running inference on
            # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
            NAME = "cvat"
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
            NUM_CLASSES = num_c

        config = InferenceConfig()
        #config.display()

        # Create model object in inference mode.
        self.model = modellib.MaskRCNN(mode="inference", model_dir="./output", config=config)
        # Load weights trained on MS-COCO
        self.model.load_weights(model_path, by_name=True)
        self.labels_mapping = {0:'BG', 1:'cut'}

    def get_polygons(self, images, threshold):
        res = self.model.detect(images)
        result = {}
        for r in res:
            for index, c_id in enumerate(r['class_ids']):
                if c_id in self.labels_mapping.keys():
                    if r['scores'][index] >= threshold:
                        mask = r['masks'][:,:,index].astype(np.uint8)
                        contours = find_contours(mask, 0.5)

                        # KLUDGE
                        # Handles a rare "list index out of range error" for contours[0]
                        # If the contours array is empty, a dummy contour consisting of
                        # the top left pisxel is provided.

                        if not contours:
                            print('ERROR: contour list is empty.')
                            contour = np.array([[1.0,1.0],[1.0,0.0],[0.0,0.0],[0.0,1.0],[1.0,1.0]])
                        else:
                            contour = contours[0]
                            # print(f'contour ({type(contour)}): {contour}')

                        # end of KLUDGE

                        contour = np.flip(contour, axis=1)
                        contour = approximate_polygon(contour, tolerance=2.5)
                        segmentation = contour.ravel().tolist()
                        label = self.labels_mapping[c_id]
                        if label not in result:
                            result[label] = []
                        result[label].append(segmentation)
        return result


    @staticmethod
    def process_polygons(polygons, boxes):
        """
           Check if any point of the polygon falls into any of coconot palms except for dead/non_recoverable.
        """
        def _check_inside_boxes(polygon, boxes):
            for point in polygon:
                for label, bxes in boxes.items():
                    for box in bxes:
                        if point[0] > box[0] and point[0] < box[2] and point[1] > box[1] and point[1] < box[3] and label not in ['dead','non_recoverable']:
                            # point is inside rectangle
                            return True
            return False

        result = {}
        for label_m, polys in polygons.items():
            for polygon in polys:
                p = [polygon[i:i+2] for i in range(0, len(polygon),2)]
                if _check_inside_boxes(p, boxes):
                    if label_m not in result:
                        result[label_m] = []
                    result[label_m].append(polygon)

        return result


def load_image_into_numpy(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def draw_instances(frame, boxes, masks):
    colors = {'zero':(0,255,0), 'light':(0,0,255),'medium':(255,0,0),'high':(120,120,0),'non_recoverable':(0,120,120),'cut':(0,0,0)}
    #draw boxes
    for label, bxes in boxes.items():
        for box in bxes:
            cv2.rectangle(frame, (box[0],box[1]), (box[2],box[3]), colors[label], 5)
    #draw polygons
    for label, polygons in masks.items():
        for polygon in polygons:
            p = [polygon[i:i+2] for i in range(0, len(polygon),2)]
            pts = np.array(p, np.int32)
            pts = pts.reshape((-1,1,2))
            cv2.polylines(frame, [pts], True, (0,255,255),5)
    return frame

def get_labels(classes_csv, type="od"):
    labels = []
    with open(classes_csv, "r") as f:
        data = f.readlines()
        # slogger.glob.info("class file data {}".format(data))
        for line in data[1:]:
            if type == "maskrcnn":
                if "," not in line:
                    continue
                # slogger.glob.info("classes line {}".format(line))
                label, num = line.strip().split(',')
                labels.append(('label', [('name', line.strip())]))
            else:
                if "label" not in line:
                    labels.append(('label', [('name', line.strip())]))
    return labels

time: 6.63 ms (started: 2021-05-02 11:19:48 +10:00)


In [7]:
# NEW CODE

# Initialization
################

# Get a sorted list of image files

# Intialize other variables

image_files = sorted(glob.glob(IMAGE_FILE_PATH))
num_frames = len(image_files)

labels_from_csv = get_labels(CLASSES_CVAT, CLASSES_TYPE)
final_result = {'meta':{'task': OrderedDict([('id',str(TASK_ID)),
                                             ('name',str(TASK_NAME)),
                                             ('size',str(num_frames)),
                                             ('mode','interpolation'),
                                             ('start_frame', str(0)),
                                             ('stop_frame', str(num_frames-1)),
                                             ('z_order',"False"),
                                             ('labels', labels_from_csv)])},
                'frames':[]}

if TYPE == "both":
    od_model = ObjectDetection(OD_MODEL)
    seg_model = Segmentation(MASK_MODEL)
elif TYPE == "classes":
    od_model = ObjectDetection(OD_MODEL)
elif TYPE == "v_shape":
    seg_model = Segmentation(MASK_MODEL)
    
labels_mapping_od = {1:'zero',2:'light',3:'medium',4:'high',5:'non_recoverable'}

# Get size of first image in list. It is assumed that all images are the same size.

frame = cv2.imread(image_files[0])
frame_height, frame_width, channels = frame.shape

height, width = frame_height, frame_width

# Process image files
#####################

frame_no = 0
for image_file in image_files:
    frame_no += 1
    print(f'Image {frame_no} of {num_frames}')
    frame = cv2.imread(image_file)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_np_expanded = np.expand_dims(img, axis=0)

    od_result = {}
    result = {}
    if TYPE == "both" or TYPE == "classes":
        # run detection
        boxes, scores, classes, num_detections = od_model.get_detections(image_np_expanded)
        #normalize bounding boxes, also apply threshold
        od_result = ObjectDetection.process_boxes(boxes, scores, classes, labels_mapping_od, OD_THRESHOLD, width, height)
        if od_result:
            print("od", od_result)
            shapes = []
            for label, boxes in od_result.items():
                for box in boxes:
                    shapes.append({'type':'rectangle','label':label,'occluded':0,'points':box})
            final_result['frames'].append({'frame':frame_no, 'width':frame_width, 'height':frame_height, 'shapes':shapes})
    if TYPE == "both" or TYPE == "v_shape":
        # run segmentation
        result = seg_model.get_polygons([img], MASK_THRESHOLD)
        print("Result before processing: ", result)
        if TYPE == "both" or TYPE == "classes":
            # filter out false positives if boxes are available
            result = Segmentation.process_polygons(result, od_result)
            print("Result after processing: ", result)
        if result:
            shapes = []
            for label, polygons in result.items():
                for polygon in polygons:
                    shapes.append({'type':'polygon','label':label,'occluded':0,'points':polygon})
            frame_exists = False
            for frame_ in final_result['frames']:
                if frame_['frame'] == frame_no:
                    break
            if frame_exists:
                final_result['frames']['shapes'].extend(shapes)
            else:
                final_result['frames'].append({'frame':frame_no, 'width':frame_width, 'height':frame_height, 'shapes':shapes})

#frame = draw_instances(frame, od_result, result)
dump_as_cvat_annotation(open(OUTPUT_XML_PATH, "w"), final_result)

print('FINISHED')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


Image 1 of 180


...

Result before processing:  {}
Result after processing:  {}
Image 177 of 180


Result before processing:  {}
Result after processing:  {}
Image 178 of 180
od {'light': [[1623, 671, 1909, 980]]}


Result before processing:  {'cut': [[469.0, 615.5, 456.5, 615.0, 466.5, 611.0, 462.5, 608.0, 461.5, 593.0, 451.0, 583.5, 447.0, 589.5, 443.5, 582.0, 447.5, 578.0, 451.5, 527.0, 457.0, 518.5, 481.5, 523.0, 478.5, 527.0, 482.5, 536.0, 490.0, 543.5, 494.5, 538.0, 495.5, 550.0, 499.5, 552.0, 507.5, 572.0, 506.5, 582.0, 494.5, 599.0, 469.0, 615.5], [1519.0, 438.5, 1488.0, 428.5, 1486.5, 422.0, 1498.0, 408.5, 1522.0, 401.5, 1525.5, 404.0, 1524.5, 413.0, 1529.5, 425.0, 1526.5, 434.0, 1519.0, 438.5]]}
Result after processing:  {}
Image 179 of 180
od {'light': [[1623, 684, 1912, 975]]}


Result before processing:  {}
Result after processing:  {}
Image 180 of 180


Result before processing:  {'cut': [[463.0, 609.5, 454.0, 609.5, 442.0, 601.5, 439.0, 604.5, 436.5, 600.0, 439.0, 586.5, 443.5, 595.0, 443.5, 583.0, 439.5, 579.0, 448.5, 559.0, 451.5, 525.0, 458.0, 511.5, 478.0, 514.5, 478.5, 519.0, 494.5, 532.0, 495.5, 553.0, 501.5, 569.0, 495.5, 591.0, 478.0, 606.5, 463.0, 609.5]]}
Result after processing:  {}
FINISHED
time: 1min 30s (started: 2021-05-02 11:19:48 +10:00)
